# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchsummary
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# Set Hyper Parameters

In [3]:
hyper_parameters = {
    "SEED":1990,
    "notebookName":"Baseline3",
    "nepochs":10000,
    "batch_size":1025,
    "learning_rate":0.1,
    "window_size":500,
    "max_patience_count":25
}

In [4]:
torch.manual_seed(hyper_parameters["SEED"])

In [5]:
f"./models/{hyper_parameters['notebookName']}"

'./models/Baseline3'

In [6]:

PATH = Path(f"./models/{hyper_parameters['notebookName']}")
if os.path.isdir(PATH):
    dir_list = os.listdir(PATH)
    num_files = 0
    while True:
        if os.path.isfile(str(PATH / f"{num_files}")):
            print(num_files)
            num_files += 1
        else:
            break
else:
    os.mkdir(PATH)
    num_files = 0
num_files

0
1
2
3


4

# Set Path

In [7]:
data_dir = Path("../input/google-smartphone-decimeter-challenge")

# Help Functions

In [8]:
def calc_haversine(lat1, lon1, lat2, lon2):
    
    lat1=lat1 % 360
    lon1=lon1 % 360
    lat2=lat2 % 360
    lon2=lon2 % 360
    
    lat1, lat2, lon1, lon2 = map(np.radians, [lat1, lat2, lon1, lon2])
    
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(a ** 0.5)
        
    dist = 6_367_000 * c
    return dist

In [9]:
def check_score_np(predict:torch.Tensor, target:torch.Tensor):
    m = []
    predict = predict.detach().numpy()
    target = target.detach().numpy()
    for i in range(predict.shape[0]):
        temp = calc_haversine(predict[i,0], predict[i,1], target[i,0], target[i,1])
        m.append(temp)
    
    m = np.array(m)
    score = (np.percentile(m, 50) + np.percentile(m, 95))/2
    
    return score

# Load Data

In [10]:
df_train_default = pd.read_pickle(str(data_dir / "gsdc_extract_train.pkl.gzip"))

In [11]:
df_test = pd.read_pickle(str(data_dir / "gsdc_extract_test.pkl.gzip"))

In [12]:
for col in df_train_default.columns:
    print(col)

collectionName
phoneName
millisSinceGpsEpoch
latDeg
lngDeg
heightAboveWgs84EllipsoidM
phone
timeSinceFirstFixSeconds
hDop
vDop
speedMps
courseDegree
t_latDeg
t_lngDeg
t_heightAboveWgs84EllipsoidM
constellationType
svid
signalType
receivedSvTimeInGpsNanos
xSatPosM
ySatPosM
zSatPosM
xSatVelMps
ySatVelMps
zSatVelMps
satClkBiasM
satClkDriftMps
rawPrM
rawPrUncM
isrbM
ionoDelayM
tropoDelayM
utcTimeMillis
TimeNanos
LeapSecond
FullBiasNanos
BiasNanos
BiasUncertaintyNanos
DriftNanosPerSecond
DriftUncertaintyNanosPerSecond
HardwareClockDiscontinuityCount
Svid
TimeOffsetNanos
State
ReceivedSvTimeNanos
ReceivedSvTimeUncertaintyNanos
Cn0DbHz
PseudorangeRateMetersPerSecond
PseudorangeRateUncertaintyMetersPerSecond
AccumulatedDeltaRangeState
AccumulatedDeltaRangeMeters
AccumulatedDeltaRangeUncertaintyMeters
CarrierFrequencyHz
MultipathIndicator
ConstellationType
AgcDb
BasebandCn0DbHz
FullInterSignalBiasNanos
FullInterSignalBiasUncertaintyNanos
SatelliteInterSignalBiasNanos
SatelliteInterSignalBiasUnc

# Dataloader

In [13]:
df_train_default['phone'].value_counts()

2021-04-22-US-SJC-1_Pixel4             2890
2021-04-22-US-SJC-1_SamsungS20Ultra    2826
2020-09-04-US-SF-2_Mi8                 2500
2021-04-29-US-SJC-2_SamsungS20Ultra    2370
2020-09-04-US-SF-2_Pixel4              2349
                                       ... 
2021-01-05-US-SVL-2_Pixel4XL           1193
2020-06-05-US-MTV-1_Pixel4XLModded     1123
2021-04-26-US-SVL-1_Mi8                1036
2021-04-26-US-SVL-1_Pixel5             1034
2020-05-14-US-MTV-2_Pixel4XLModded      577
Name: phone, Length: 73, dtype: int64

In [14]:
def CustomTrainValidSplit(df:pd.DataFrame, valid_size):
    phones = df['phone'].unique()
    
    valid_num = int(len(phones) * valid_size)
    train_num = len(phones) - valid_num
    
    indexes = np.array(range(len(phones)))
    indexes = np.random.choice(indexes, len(indexes))
    
    df_train = []
    for phone in phones[indexes[:train_num]]:
        df_train.append(df[df['phone'] == phone])
    df_train = pd.concat(df_train)
    
    df_valid = []
    for phone in phones[indexes[train_num:-1]]:
        df_valid.append(df[df['phone'] == phone])
    df_valid = pd.concat(df_valid)
    
    return df_train.reset_index().drop(columns = 'index'), df_valid.reset_index().drop(columns = 'index')
    
df_train, df_valid = CustomTrainValidSplit(df_train_default, valid_size = 0.1)
print(df_train.shape, df_valid.shape)
    

(120642, 138) (10779, 138)


In [15]:
df_train

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,timeSinceFirstFixSeconds,hDop,vDop,...,BiasZMps2_Scaled,UncalMagXMicroT_Scaled,UncalMagYMicroT_Scaled,UncalMagZMicroT_Scaled,BiasXMicroT_Scaled,BiasYMicroT_Scaled,BiasZMicroT_Scaled,yawDeg_Scaled,rollDeg_Scaled,pitchDeg_Scaled
0,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160577000,37.334587,-121.899431,-4.10,2021-04-22-US-SJC-1_SamsungS20Ultra,327.00,1.8,0.0,...,-2.065078,-0.684273,-0.433202,-0.308914,-0.942276,-0.096116,-0.448045,-0.976377,0.600596,-0.508054
1,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160578000,37.334564,-121.899395,-5.95,2021-04-22-US-SJC-1_SamsungS20Ultra,328.00,1.8,0.0,...,-2.065078,-0.675184,-0.408991,-0.313342,-0.942276,-0.096116,-0.448045,-0.976377,0.600596,-0.508054
2,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160579000,37.334579,-121.899414,-5.03,2021-04-22-US-SJC-1_SamsungS20Ultra,329.00,1.8,0.0,...,-2.065078,-0.678214,-0.442886,-0.306257,-0.942276,-0.096116,-0.448045,-0.976377,0.600596,-0.508054
3,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160580000,37.334573,-121.899406,-4.78,2021-04-22-US-SJC-1_SamsungS20Ultra,330.00,1.8,0.0,...,-2.065078,-0.678214,-0.431588,-0.300057,-0.942276,-0.096116,-0.448045,-0.976377,0.600596,-0.508054
4,2021-04-22-US-SJC-1,SamsungS20Ultra,1303160581000,37.334588,-121.899438,-9.13,2021-04-22-US-SJC-1_SamsungS20Ultra,331.00,1.8,0.0,...,-2.065078,-0.679729,-0.410605,-0.308028,-0.942276,-0.096116,-0.448045,-0.976377,0.600596,-0.508054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120637,2021-04-22-US-SJC-1,Pixel4,1303163462446,37.334590,-121.899411,-15.98,2021-04-22-US-SJC-1_Pixel4,3212.45,2.9,0.0,...,0.484243,0.077927,-0.559201,-1.163001,-0.246170,-0.066593,-0.661824,-0.498000,0.561373,-0.565837
120638,2021-04-22-US-SJC-1,Pixel4,1303163463446,37.334579,-121.899403,-17.10,2021-04-22-US-SJC-1_Pixel4,3213.45,2.9,0.0,...,0.484243,0.077927,-0.559201,-1.163001,-0.246170,-0.066593,-0.661824,-0.498000,0.561373,-0.565837
120639,2021-04-22-US-SJC-1,Pixel4,1303163464446,37.334589,-121.899423,-17.23,2021-04-22-US-SJC-1_Pixel4,3214.45,2.9,0.0,...,0.484243,0.077927,-0.559201,-1.163001,-0.246170,-0.066593,-0.661824,-0.498000,0.561373,-0.565837
120640,2021-04-22-US-SJC-1,Pixel4,1303163465446,37.334621,-121.899401,-16.86,2021-04-22-US-SJC-1_Pixel4,3215.45,2.9,0.0,...,0.484243,0.077927,-0.559201,-1.163001,-0.246170,-0.066593,-0.661824,-0.498000,0.561373,-0.565837


In [16]:
def GetWindows(idx, window_size):
    index = np.array([])
    if idx < window_size:
        index = np.concatenate([np.zeros(window_size - idx-1), np.array(range(idx+1))])
        pass
    else:
        index = np.array(range(idx-window_size+1, idx+1))
    return index.astype(int)

def GetWindowsWithRatio(idx, max_idx, window_size, window_ratio = 1.):
    left_index = np.array([])
    right_index = np.array([])
    
    left_size = int(round(window_size * window_ratio))
    right_size = window_size - left_size
    if idx - left_size< 0:
        left_index = np.concatenate([np.zeros(left_size - idx-1), np.array(range(idx+1))])
    else:
        left_index = np.array(range(idx-left_size, idx+1))
    
    if idx + right_size> max_idx:
        right_index = np.concatenate([np.array(range(idx+1, max_idx+1)), (max_idx)*np.ones(right_size - idx - 1)])
    else:
        right_index = np.array(range(idx+1, idx + right_size))
    
    index = np.concatenate([left_index, right_index])
    
    if index.shape[0] < window_size:
        if idx > np.percentile(index, window_ratio * 100):
            if index[-1] == max_idx:
                addtional_index = index[-1]
            else:
                addtional_index = index[-1] + 1
            index = np.concatenate([index, np.array([addtional_index])])
        
    return index.astype(int)


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df:pd.DataFrame, 
                 features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM'], 
                 labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM'],
                 window_size = 100,
                 train = True,
                device = 'cpu'):
        self.df = df
        self.features = features
        self.labels = labels
        self.len = df.shape[0]
        self.window_size = window_size
        self.train = train
        self.device = device
        
        self.data = self.df[features].astype(float).values
        if train == True:
            self.true = self.df[labels].astype(float).values
        else:
            self.true = []
        self.phone = self.df['phone'].values
        self.millisSinceGpsEpoch = self.df['millisSinceGpsEpoch'].values
        
        self.start_index_by_phone = dict()
        self.length_by_phone = dict()
        
        for phone in set(self.phone):
            start_index = np.where(self.phone == phone)[0][0]
            self.start_index_by_phone[phone] = start_index
            self.length_by_phone[phone] = (self.phone == phone).sum().astype('int64')
            
        
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        # 필요하다면 random crop 코드도 추가(랜덤한 위치에서 일정 비율의 데이터를 0으로 초기화)
        phone = self.phone[idx]
        start_index = self.start_index_by_phone[phone]
        
#         window_index = GetWindowsWithRatio(idx - start_index, self.length_by_phone[phone], self.window_size, 1) + start_index
        window_index = GetWindows(idx - start_index, self.window_size) + start_index
        data = self.data[window_index, :].transpose(1, 0)
        
            
        indx = [self.phone[idx], self.millisSinceGpsEpoch[idx]]
        
        # data shape : num_of_features X window_size
        # true shape : num_of_labels X 1
        data = torch.Tensor(data)
        
        if self.train is False:
            true = []
        else:
            true = self.true[idx]
            true = torch.Tensor(true.astype(float))
        
        return data, true, indx
    


In [17]:
features = ['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM',
           'dlatDeg', 'dlngDeg', 'dheight', 
            'dlatDeg_Scaled',
            'dlngDeg_Scaled',
            'dheight_Scaled',
            'xSatPosM_Scaled',
            'ySatPosM_Scaled',
            'zSatPosM_Scaled',
            'xSatVelMps_Scaled',
            'ySatVelMps_Scaled',
            'zSatVelMps_Scaled',
            'UncalGyroXRadPerSec_Scaled',
            'UncalGyroYRadPerSec_Scaled',
            'UncalGyroZRadPerSec_Scaled',
            'DriftXRadPerSec_Scaled',
            'DriftYRadPerSec_Scaled',
            'DriftZRadPerSec_Scaled',
            'UncalAccelXMps2_Scaled',
            'UncalAccelYMps2_Scaled',
            'UncalAccelZMps2_Scaled',
            'BiasXMps2_Scaled',
            'BiasYMps2_Scaled',
            'BiasZMps2_Scaled',
            'UncalMagXMicroT_Scaled',
            'UncalMagYMicroT_Scaled',
            'UncalMagZMicroT_Scaled',
            'BiasXMicroT_Scaled',
            'BiasYMicroT_Scaled',
            'BiasZMicroT_Scaled',
            'yawDeg_Scaled',
            'rollDeg_Scaled',
            'pitchDeg_Scaled']
labels = ['t_latDeg', 't_lngDeg', 't_heightAboveWgs84EllipsoidM']

train_data = CustomDataset(df_train, features = features, labels = labels, window_size = hyper_parameters['window_size'], device = device)
valid_data = CustomDataset(df_valid, features = features, labels = labels, window_size = hyper_parameters['window_size'], device = device)
test_data = CustomDataset(df_test, features = features, labels = labels, window_size = hyper_parameters['window_size'], train = False, device = device)

In [18]:
train_loader = DataLoader(train_data, batch_size = hyper_parameters['batch_size'], shuffle = True)
valid_loader = DataLoader(valid_data, batch_size = hyper_parameters['batch_size'], shuffle = False)
test_loader = DataLoader(test_data, batch_size = hyper_parameters['batch_size'], shuffle = False)

# Build Model

In [19]:
class GPSLossScore(nn.Module):
    def __init__(self):
        super().__init__()
        self.EARTH_RADIUS = 6_367_000
        self.mse = nn.MSELoss()
        pass
    
    def forward(self, predict:torch.Tensor, target:torch.Tensor):
        
        loss = (0.5) * self.gps_loss(predict, target) + (0.5) * self.EARTH_RADIUS * self.mse(predict, target)
        return loss
    
    def torch_haversine(self,lat1, lon1, lat2, lon2):
    
        lat1=lat1 % 360
        lon1=lon1 % 360
        lat2=lat2 % 360
        lon2=lon2 % 360

        lat1, lat2, lon1, lon2 = map(torch.deg2rad, [lat1, lat2, lon1, lon2])

        dlat = (lat2 - lat1)
        dlon = (lon2 - lon1)

        a = torch.sin(dlat / 2.0)**2 + torch.cos(lat1) * torch.cos(lat2) * (torch.sin(dlon / 2.0)**2)
        c = 2 * torch.arcsin(a ** 0.5)

        dist = self.EARTH_RADIUS * c

        return dist

    def gps_loss(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        loss = dist.mean()

        return loss

    def gps_score(self, predict:torch.Tensor, target:torch.Tensor):
        dist = self.torch_haversine(predict[:,0], predict[:,1], target[:,0], target[:,1])

        score = (torch.quantile(dist, 0.5) + torch.quantile(dist, 0.95))/2

        return score
    

In [20]:
class OneConv(nn.Module):
    def __init__(self, input_size = 3, output_size = 3, filter_size = 3):
        super().__init__()
        
        self.filter = nn.Conv1d(input_size, output_size, filter_size, padding = filter_size//2)
        self.skip_filter = nn.Conv1d(input_size, output_size, 1, padding = 0)
        self.batch_norm = nn.BatchNorm1d(output_size)
        
    def forward(self, x):
        skip = self.skip_filter(x)
        filt = F.relu(self.filter(x))
        x = filt + skip
        x = self.batch_norm(x)
        return x
    
class StackWidth(nn.Module):
    def __init__(self, input_size = 3, output_size = 3, filter_size_list = [1,3,5]):
        super().__init__()
        self.skip = nn.Conv1d(input_size, output_size * len(filter_size_list), 1, padding = 0)
        
        self.conv1 = OneConv(input_size, output_size, filter_size_list[0])
        self.conv3 = OneConv(input_size, output_size, filter_size_list[1])
        self.conv5 = OneConv(input_size, output_size, filter_size_list[2])
        
        self.conv_cat = OneConv(output_size  * len(filter_size_list), output_size, 1)
        
    def forward(self, x):
        x0 = self.skip(x)
        
        x1 = self.conv1(x)
        x3 = self.conv3(x)
        x5 = self.conv5(x)
        
        x = torch.cat([x1,x3,x5], axis = 1)
        x = x + x0
        x = self.conv_cat(x)
        
        return x
    
class StackHeight(nn.Module):
    def __init__(self, _input_size = 3, output_size_list = [3, 5, 7]):
        super().__init__()
        input_size = _input_size
        self.skip = nn.Conv1d(input_size, output_size_list[-1], 1, padding = 0)
        
        self.conv3 = StackWidth(input_size, output_size_list[0], [1,3,5])
        self.conv5 = StackWidth(output_size_list[0], output_size_list[1], [1,3,5])
        self.conv7 = StackWidth(output_size_list[1], output_size_list[2], [1,3,5])
        
    def forward(self, x):
        x0 = self.skip(x)
        
        x = self.conv3(x)
        x = self.conv5(x)
        x = self.conv7(x)
        
        x = x + x0
        x = F.relu(x)
        
        return x
    

In [21]:

class BaseModel(nn.Module):
    def __init__(self, input_size = (100, 3), output_size = 3):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        
        self.conv1 = StackHeight(input_size[1], [36, 64, 64])
        self.maxpool1 = nn.MaxPool1d(2, 2)
        self.batch1 = nn.BatchNorm1d(64)
        
        self.conv2 = StackHeight(64, [32, 16, 8])
        self.maxpool2 = nn.MaxPool1d(2, 2)
        self.batch2 = nn.BatchNorm1d(8)
        
        self.conv3 = StackHeight(8, [8, 8, 8])
        self.maxpool3 = nn.MaxPool1d(2, 2)
        self.batch3 = nn.BatchNorm1d(8)
        
        self.fc1 = nn.Linear((input_size[0]//8)*8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 3)
        
        self.drop03 = nn.Dropout(0.3)
        self.drop05 = nn.Dropout(0.5)
        self.drop06 = nn.Dropout(0.6)
        
    def forward(self, x):
        input_size = self.input_size
        
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.batch1(x)
        
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.batch2(x)
        
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.batch3(x)
        
        x = x.view(-1, (input_size[0]//8)*8)
        
        x = self.drop06(x)
        x = F.relu(self.fc1(x))
        x = self.drop05(x)
        x = F.relu(self.fc2(x))
        x = self.drop05(x)
        x = F.relu(self.fc3(x))
        x = self.drop05(x)
        x = F.relu(self.fc4(x))
        x = self.drop03(x)
        x = self.fc5(x)
        
        return x

In [22]:
model = BaseModel((hyper_parameters['window_size'], len(features)), len(labels))
model.to(device)

# loss_func = nn.SmoothL1Loss()
loss_func = GPSLossScore()
optimizer = optim.Adam(model.parameters(), lr = hyper_parameters['learning_rate'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                mode = 'min',
                                                factor = 0.9,
                                                patience = 10)

# Fit and Validate

In [23]:
def train(epoch, dataloader):
    model.train()  # 신경망을 학습 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행
    predict = []
    ground = []
    
    prog_bar = tqdm(dataloader, position = 1, desc = 'Train')
    status_log = tqdm(total=0, position = 2, bar_format='{desc}')
    for data, targets, _ in prog_bar:
        
        data = data.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()  # 경사를 0으로 초기화
        outputs = model(data)  # 데이터를 입력하고 출력을 계산
        loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
        
        loss.backward()  # 오차를 역전파 계산
        optimizer.step()  # 역전파 계산한 값으로 가중치를 수정
        
        predict.append(outputs)
        ground.append(targets)
        
        status_log.set_description_str(f"training status: [{loss_func(outputs, targets)}/{loss_func.gps_score(outputs, targets)}]")

    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    
    scheduler.step(loss)
    
    return loss, meas

In [24]:
def valid(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행
    predict = []
    ground = []
    prog_bar = tqdm(dataloader, position = 1, desc = 'Valid')
    status_log = tqdm(total=0, position = 2, bar_format='{desc}')
    
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, targets, _ in prog_bar:
            
            data = data.to(device)
            targets = targets.to(device)
            
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            loss = loss_func(outputs, targets)  # 출력과 훈련 데이터 정답 간의 오차를 계산
            
            predict.append(outputs)
            ground.append(targets)
            
            status_log.set_description_str(f"valid status: [{loss_func(outputs, targets)}/{loss_func.gps_score(outputs, targets)}]")
            
    # 정확도 출력
    predict = torch.cat(predict,axis = 0)
    ground = torch.cat(ground,axis = 0)
    
    loss = loss_func(predict, ground)
    meas = loss_func.gps_score(predict, ground)
    return loss, meas

In [25]:
def test(dataloader):
    model.eval()  # 신경망을 추론 모드로 전환
    
    output_list = []
    with torch.no_grad():  # 추론 과정에는 미분이 필요없음
        for data, _, index in tqdm(dataloader):
            data = data.to(device)
            outputs = model(data)  # 데이터를 입력하고 출력을 계산
            
            df_temp = pd.DataFrame()
            df_temp['phone'] = index[0]
            df_temp['millisSinceGpsEpoch'] = index[1]
            df_temp[['latDeg', 'lngDeg', 'heightAboveWgs84EllipsoidM']] = outputs.to('cpu').numpy()
            output_list.append(df_temp)
    
    predicts = pd.concat(output_list)
    return predicts
            

In [ ]:
train_loss_list = []
train_meas_list = []
valid_loss_list = []
valid_meas_list = []

patience_count = 0
min_valid_meas = np.inf
checkpoint_name = ""

for epoch in range(hyper_parameters['nepochs']):
    train_loss, train_meas = train(epoch, train_loader)
    valid_loss, valid_meas = valid(valid_loader)
    
    if valid_meas < min_valid_meas:
        min_valid_meas = valid_meas
        checkpoint_name = f"./models/{hyper_parameters['notebookName']}/model-{num_files}_checkpoint-{min_valid_meas}.pth"
        torch.save(model.state_dict(), checkpoint_name)
    else:
        patience_count+=1
        if(patience_count > hyper_parameters['max_patience_count']):
            break
    
    train_loss_list.append(train_loss)
    train_meas_list.append(train_meas)
    valid_loss_list.append(valid_loss)
    valid_meas_list.append(valid_meas)
   
    
    print(f"{epoch+1}/{hyper_parameters['nepochs']}: train[{train_loss:.6f}/{train_meas}], valid[{valid_loss}/{valid_meas}]")

history = dict()
history['train_loss'] = train_loss_list
history['train_meas'] = train_meas_list
history['valid_loss'] = valid_loss_list
history['valid_meas'] = valid_meas_list


Train:   0%|          | 0/118 [00:00<?, ?it/s]

Valid:   0%|          | 0/11 [00:00<?, ?it/s]

1/10000: train[1296775046496256.000000/12844692.0], valid[35449081856.0/12846848.0]


Train:   0%|          | 0/118 [00:00<?, ?it/s]

Valid:   0%|          | 0/11 [00:00<?, ?it/s]

2/10000: train[29908004864.000000/12836947.0], valid[35378311168.0/12830710.0]


Train:   0%|          | 0/118 [00:00<?, ?it/s]

In [ ]:
plt.figure(figsize = (16,6))
plt.subplot(2,1,1)
plt.plot(history['train_loss'], label = 'train')
plt.plot(history['valid_loss'], label = 'valid')
plt.ylabel('loss')
plt.yscale('log')

plt.subplot(2,1,2)
plt.plot(history['train_meas'], label = 'train')
plt.plot(history['valid_meas'], label = 'valid')
plt.ylabel('meas')
plt.yscale('log')

#  Output

In [ ]:
# Load submission sample
submission = pd.read_csv(str(data_dir / "sample_submission.csv"))
print(submission.shape)
submission.head()

In [ ]:
model.load_state_dict(torch.load(checkpoint_name))
torch.save(model.state_dict(), f"./models/{hyper_parameters['notebookName']}/model-{num_files}.pth")

In [ ]:
predict = test(test_loader)
print(predict.shape)
predict.head()

In [ ]:
submission = submission[['phone', 'millisSinceGpsEpoch']].merge(predict[['phone', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']]
                                                                , on = ['phone', 'millisSinceGpsEpoch'])
print(submission.shape)
submission.head()

In [ ]:
submission.to_csv(f"./models/{hyper_parameters['notebookName']}/result-{num_files}esult.csv", index = False)
pd.DataFrame([]).to_csv(PATH / f"{num_files}")